In [ ]:
import imagej
import numpy as np
import glob
import cv2 
import matplotlib.pyplot as plt 
import pandas as pd 
from skimage.filters import roberts, sobel, scharr, prewitt
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def crop_image(img, size):
    h, w = img.shape[:2]

    center_h = h // 2
    center_w = w // 2

    crop_h, crop_w = size
    
    top = center_h - crop_h // 2
    left = center_w - crop_w // 2
    bottom = top + crop_h
    right = left + crop_w
    
    cropped_img = img[top:bottom, left:right]
    
    return cropped_img

def bandpass_filter (img, low_cut=3, high_cut=40, tolerance=5):
    # Start ImageJ
    ij = imagej.init()
    
    # Convert the image to a 32-bit float
    img = img.astype(np.float32)
    
    # Apply the bandpass filter using ImageJ
    filtered_img = ij.py.run_script("run('Bandpass Filter...', 'filter_large=" + str(high_cut) + " filter_small=" + str(low_cut) + " suppress=None tolerance=" + str(tolerance) + "');", img)
    
    # Shutdown ImageJ
    ij.py.shutdown()
    
    return filtered_img

def register_image(reference_img, image):
    ij = imagej.init()

    imp1 = ij.py.to_java(reference_img)
    imp2 = ij.py.to_java(image)

    plugin_path = "Plugins/Registration/Register Virtual Stack Slices.ijm"
    plugin_file = ij.py.gateways.get_single_file(plugin_path)
    plugin = ij.py.plugins().get_plugin(plugin_file)

    plugin.setup(ij.py.get_opener(), imp1, imp2)
    plugin.show_dialog()
    plugin.run(ij.py.new_ij())

    registered_imp = plugin.get_output()
    registered_img = np.array(ij.py.from_java(registered_imp.getProcessor().getPixels()), dtype=np.uint8)

    ij.py.shutdown()

    return registered_img

def feature_extractor(img):
    df = pd.DataFrame()

    flattened_img = img.reshape(-1)
    df['Original Image'] = flattened_img

    # Gabor filter 
    num = 1
    kernels = []
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
                
                    gabor_label = 'Gabor' + str(num)
                    ksize=9
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                    kernels.append(kernel)
                    fimg = cv2.filter2D(flattened_img, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img  
                    num += 1
       
    # Edge filters 
    # Canny edge 
    edges = cv2.Canny(img, 100,200)   
    edges1 = edges.reshape(-1)
    df['Canny Edge'] = edges1 

    # Roberts edge
    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df['Roberts'] = edge_roberts1

    # Sobel
    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df['Sobel'] = edge_sobel1

    # Scharr
    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df['Scharr'] = edge_scharr1

    # Prewitt
    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['Prewitt'] = edge_prewitt1

    # Gaussian with sigma=3
    from scipy import ndimage as nd
    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df['Gaussian s3'] = gaussian_img1

    # Gaussian with sigma=7
    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df['Gaussian s7'] = gaussian_img3

    # Median with sigma=3
    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    df['Median s3'] = median_img1

    # Variance with size=3
    variance_img = nd.generic_filter(img, np.var, size=3)
    variance_img1 = variance_img.reshape(-1)
    df['Variance s3'] = variance_img1  

    return df

In [ ]:
# Step_1: Preprocessing the images dataset and generating the features bank

path = 'training/*.*'
path_to_save = 'savings/'
reference = cv2.imread('reference.png') 
image_dataset = pd.DataFrame()  

for one_file in glob.glob(path):
    img_name = one_file.split("\\") [-1]    
    img = cv2.imread(one_file)
    
    # Cropping the images
    cropped_pixels = 10
    size = (img.shape[0] - cropped_pixels, img.shape[1] - cropped_pixels)
    cropped_img = crop_image (img, size)
    cv2.imwrite(path_to_save + 'cropped/' + img_name, cropped_img)
    
    # Applying bandpass filter to make illumination uniform
    filtered_img = bandpass_filter (cropped_img)
    cv2.imwrite(path_to_save + 'filtered/' + img_name, filtered_img)
    
    # Register images to correct for the potential slight change, if any, 
    # in the position of the holder of the microfluidic chip during the experiment 
    registered_image = register_image (reference, filtered_img)
    
    # generating the feature banks containing varios filters:
    df = feature_extractor (registered_image)
    
    image_dataset = image_dataset.append(df)
    

# Step_2: Reading ground truth labels 
mask_df = pd.DataFrame()  
mask_path = "training_masks/"
  
for mask in os.listdir(mask_path):  
    
    df2 = pd.DataFrame()  
    input_mask = cv2.imread(mask_path + mask)
    label = cv2.cvtColor(input_mask,cv2.COLOR_BGR2GRAY)
    
    label_values = label.reshape(-1)
    df2['Label_Value'] = label_values
    df2['Mask_Name'] = mask
    
    mask_df = mask_df.append(df2)  
  
 
# Step_3: Concatenate dataframes and splitting the data 
dataset = pd.concat([image_dataset, mask_df], axis=1)

X = dataset.drop(labels = ["Image_Name", "Mask_Name", "Label_Value"], axis=1) 

Y = dataset["Label_Value"].values 

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

# Step_4: Training the model  

model = RandomForestClassifier(n_estimators = 50, random_state = 42)
model.fit(X_train, y_train)
   
# Step_5: check model accuracy 

prediction_test = model.predict(X_test)
##Check accuracy on test dataset. 
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_test))

# IoU 


    